# Progetto - Spotify API

### Progetto di Francesco Petrosino X81000533


## Introduzione

Questo programma, grazie alle informazioni che è possibile ricavare dalle API di Spotify, permette di creare
una playlist con album e canzoni di autori preferiti dagli utenti che la richiedono, arricchendola, tramite
opportuni algoritmi, anche con alcune delle loro canzoni più in voga del momento.
<br><br>
Prima di iniziare, è opportuno precisare che tutte le informazioni, dati e quant’altro ricavato e/o elaborato dal
programma stesso, NON legge in alcun modo tutti quei dati e informazioni che Spotify ritiene private per
l’utente (tranne quelle di chi fornisce la coppia di chiavi d’accesso per le API)

## Come funziona

Tramite la sottoscrizione all’account developer con Spotify e all’utilizzo di librerie in Python è possibile
realizzare moltissimi tool.
<br><br>
Questo progetto richiede per iniziare, una lista di utenti, iscritti semplicemente alla piattaforma e che abbiano,
non necessariamente, creato in precedenza delle loro playlist pubbliche.<br>
Dopo aver definito il gruppo di utenti, il tool procederà alla raccolta dei dati, memorizzando informazioni
come: nome utente, nomi delle playlist, le relative canzoni inserite, e molto altro.
Successivamente, durante la fase dell’analisi dei dati, grazie al supporto di alcune librerie, come pandas e
numpy, e all’implementazione di un algoritmo di uno dei metodi di sistemi di raccomandazione dei dati,
verranno processate tutte le informazioni raccolte nel precedente step.<br>
Infine, verrà creata una playlist con una parte dei dati elaborati dall’algoritmo, e per rendere più creativa e
unica la playlist, tramite le informazioni in possesso di generi e autori, verrà effettuata una ricerca sulla
piattaforma stessa di alcune delle canzoni più in voga del momento, per inserirle dunque sulla playlist.

## Raccolta dei dati

### Setup

Esegui questo comando nel tuo terminale: `py -m pip install --upgrade -r .\requirements.txt`

In [1]:
# Import delle librerie

import time
import pandas
import numpy
import sklearn
import json
import spotipy
from operator import length_hint
from datetime import datetime

from spotipy.oauth2 import SpotifyOAuth
from user import User
from group import Group

import main


In [2]:
# Variabili globali

# client_id = "YOUR_CLIENT_ID"
# client_secret = "YOUR_CLIENT_SECRET"
# redirect_uri = "YOUR_REDIRECT_URI"
client_id = "b48898f561ab4533b7396439e6610ff4"
client_secret = "bbf061893e8f4ff8924eed41c39c6a90"
redirect_uri = "http://localhost:8080"

# Il token servirà per..
scope = [
    "user-library-read",
    "user-top-read",
    "user-follow-read",
    "user-read-private",
    "user-read-recently-played",
    "user-read-playback-state",
    "user-modify-playback-state",
    "user-read-playback-position",
    "streaming",
    "app-remote-control",
    "playlist-read-private",
    "playlist-modify-private",
    "playlist-modify-public",
    "playlist-read-collaborative"
]

# Autenticazione

auth_manager = SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri=redirect_uri,
    scope=scope
)

sp = spotipy.Spotify(auth_manager=auth_manager)


Una volta ottenuto il token per poter richiamare le API di Spotify, sarà possibile iniziare la spiegazione e la raccolta dei dati, per il conseguimento del progetto.

### Creazione del gruppo di utenti

Nello step successivo, viene illustrato come verranno raccolte le informazioni, relative agli utenti e le relative playlist.<br>
Sono state definite due classi di comodo che ci consentiranno di semplificare la lettura e l'analisi dei dati. Queste classi sono: <b>User</b> e <b>Group</b>

#### User class

![User](img/User.png)

#### Group class

![Group1](img/Group1.png)
![Group2](img/Group2.png)
![Group3](img/Group3.png)

In [3]:
print("Creo il gruppo di utenti...")

# Istanza del gruppo di utenti
group = Group()

# Lista di username dei vari utenti
id_list = [
    '21ktslj6lkgx6wswmfwu5cwuq',
    'rcin690qmgjd32d088agw6xyr',
    '1171706601',
    '6ua7zipvurza24psy4h2rvumy',
    '21vfi35alpob2afo2vtoybthi',
    '392q67hc38jnsgjg2hqrjyuhc',
    '11135404841'
]

len = len(id_list)

# Per ogni username in lista, prendo le loro informazioni
# e le inserisco nel gruppo
for i, id in enumerate(id_list):
    # Cerco su Spotify le informazioni dell'utente
    result = sp.user(id)

    # Aggiungo l'utente al gruppo
    user = User(result)
    group.append(user)

    if i+1 < len:
        time.sleep(3)

group.print()


Creo il gruppo di utenti...
Il gruppo è composto da: 
Francesco Petrosino
Ale
Philip Giovanni Emmer
Renato
Elio Pampolini
Scalòppina
Aldo Fiorito


### Recupero le playlist e le relative tracks

Di seguito andremo a collezionare tutte le playlist e le relative tracks, tenendo traccia solamente delle informazioni più importanti, che ci consentiranno di analizzare al meglio le informazioni nello step successivo tramite un algoritmo basato su uno dei sistemi di raccomandazione

In [4]:
print("Recupero le playlists dal server...")

len = group.count()

# Per ogni utente del gruppo
# colleziono le sue playlists pubbliche
for i in range(len):
    # Prendo l'utente corrente
    user = group.get_from_list(i)

    print(user.display_name)

    # Cerco la playlist dell'utente
    results = sp.user_playlists(user=user.user_id, limit=10)

    # Colleziono la playlist dell'utente
    user.set_following_playlists(results['items'])

    if i+1 < len:
        time.sleep(5)


Recupero le playlists dal server...
Francesco Petrosino
Ale
Philip Giovanni Emmer
Renato
Elio Pampolini
Scalòppina
Aldo Fiorito


In [5]:
print("Creo un dataframe con le playlists...")

len = group.count()

for i in range(len):
    # Prendo l'utente corrente
    user = group.get_from_list(i)

    # Creo una lista temporanea delle playlist dell'utente corrente
    playlist_list = list(map(lambda playlist: {
        'user_id': user.user_id,
        'user_display_name': user.display_name,
        'playlist_id': playlist.get('id'),
        'playlist_name': playlist.get('name'),
        'playlist_description': playlist.get('description'),
        'playlist_uri': playlist.get('uri'),
        'playlist_public': playlist.get('public'),
        'playlist_total_tracks': playlist.get('tracks', {}).get('total')
    }, user.get_following_playlists()))

    # Creo un dataframe con la playlist dell'utente corrente
    playlist_user_dataframe = pandas.DataFrame(playlist_list)

    # Concateno con il dataframe finale
    group.append_to_playlist_dataframe(playlist_user_dataframe)


Creo un dataframe con le playlists...


s:\Coding\ProgettoSMM\Progetto2\group.py:45: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_dataframe = pandas.concat([self.playlist_dataframe, dataframe], ignore_index=True)
s:\Coding\ProgettoSMM\Progetto2\group.py:45: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_dataframe = pandas.concat([self.playlist_dataframe, dataframe], ignore_index=True)
s:\Coding\ProgettoSMM\Progetto2\group.py:45: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_dataframe = pandas.concat([self.playlist_dataframe, dataframe], ignore_index=True)
s:\Coding\ProgettoSMM\Progetto2\group.py:45: Futu

In [6]:
print('Per ogni playlist cerco le relative tracks')

# Per ogni playlist collezionata, cerco le relative tracks
for index, playlist in group.get_playlist_dataframe().iterrows():
    print(playlist.get('playlist_name'))

    # Prendo le tracks in una playlist
    results = sp.playlist_items(playlist_id=playlist.get('playlist_id'), limit=100)

    # Converto la risposta in un json
    items_raw = json.dumps(results['items'])
    items_json = json.loads(items_raw)
    items_list = list(items_json)

    # Creo una lista temporanea delle playlist dell'utente corrente
    playlist_tracks_list = list(map(lambda track: {
        'user_id': playlist.get('user_id'),
        'user_display_name': playlist.get('user_display_name'),
        'playlist_id': playlist.get('playlist_id'),
        'playlist_name': playlist.get('playlist_name'),
        'album_id': track.get('track', {}).get('album', {}).get('id'),
        'album_name': track.get('track', {}).get('album', {}).get('name'),
        'album_release_date': track.get('track', {}).get('album', {}).get('release_date'),
        'album_total_tracks': track.get('track', {}).get('album', {}).get('total_tracks'),
        'artist_id': track.get('track', {}).get('artists', {})[0].get('id'),
        'artist_name': track.get('track', {}).get('artists', {})[0].get('name'),
        'track_disc_number': track.get('track', {}).get('disc_number'),
        'track_duration_ms': track.get('track', {}).get('duration_ms'),
        'track_explicit': track.get('track', {}).get('explicit'),
        'track_id': track.get('track', {}).get('id'),
        'track_name': track.get('track', {}).get('name'),
        'track_popularity': track.get('track', {}).get('popularity')
    }, items_list))

    # Creo un dataframe con la playlist dell'utente corrente
    playlist_tracks_dataframe = pandas.DataFrame(playlist_tracks_list)

    # Concateno con il dataframe finale
    group.append_to_playlist_tracks_dataframe(playlist_tracks_dataframe)


Per ogni playlist cerco le relative tracks
Recommend playlist - 2023-02-12
Car


s:\Coding\ProgettoSMM\Progetto2\group.py:53: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_tracks_dataframe = pandas.concat([self.playlist_tracks_dataframe, dataframe], ignore_index=True)


Guitar


s:\Coding\ProgettoSMM\Progetto2\group.py:53: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_tracks_dataframe = pandas.concat([self.playlist_tracks_dataframe, dataframe], ignore_index=True)


2021


...

s:\Coding\ProgettoSMM\Progetto2\group.py:53: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_tracks_dataframe = pandas.concat([self.playlist_tracks_dataframe, dataframe], ignore_index=True)
s:\Coding\ProgettoSMM\Progetto2\group.py:53: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_tracks_dataframe = pandas.concat([self.playlist_tracks_dataframe, dataframe], ignore_index=True)


Chill and Memes
Acoustic Mood


s:\Coding\ProgettoSMM\Progetto2\group.py:53: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_tracks_dataframe = pandas.concat([self.playlist_tracks_dataframe, dataframe], ignore_index=True)


🩹


s:\Coding\ProgettoSMM\Progetto2\group.py:53: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_tracks_dataframe = pandas.concat([self.playlist_tracks_dataframe, dataframe], ignore_index=True)


## Analisi dei dati

In questo step andremo ad analizzare tutti i dati raccolti precedentemente, tramite un algoritmo basato sul metodo <b>Content-based filtering</b> come sistema di raccomandazione, per creare infine una playlist che accomuna le preferenze degli utenti in lista.

### Content-based filtering

Il **Content-based filtering** è un metodo del sistema di raccomandazione per formulare raccomandazioni agli utenti in base alle caratteristiche o agli attributi degli elementi a cui gli utenti hanno mostrato interesse. Il sistema utilizza questi attributi per identificare elementi simili e quindi formula raccomandazioni agli utenti in base alla somiglianza.<br>
<br>
L'idea generale alla base di questo metodo è che se a un utente piace un elemento, gli piaceranno anche elementi simili, messi a fattor comune da un dataset globale come risultato ottenuto dalla raccolta dei dati. Per implementare un sistema di content-based filtering, il primo passo è estrarre "feature" o attributi chiave dagli elementi in quesitone. Queste caratteristiche possono essere tutto ciò che caratterizza l'oggetto, come il suo genere, l'artista, il numero di "mi piace", ecc.. Successivamente, viene utilizzata una misura di somiglianza, come la somiglianza del coseno, per confrontare i vettori delle caratteristiche degli elementi per identificare elementi simili. Infine, il sistema utilizza gli elementi simili identificati per formulare raccomandazioni agli utenti.<br>
<br>
Questo modello può essere diviso in due tipi principali:
- **Item-content based**: Questo metodo utilizza le caratteristiche dell'elemento per consigliare elementi simili all'utente. Ad esempio, se a un utente piace una canzone che è classificata come "rock" e ha "Jimi Hendrix" come artista, il sistema consiglierà altre canzoni che sono anche classificate come "rock" e hanno "Jimi Hendrix" come artista.
- **User-content based**: Questo metodo utilizza le funzionalità dell'utente per consigliare elementi simili alle preferenze dell'utente. Ad esempio, se a un utente piacciono i generi "rock" e "pop", il sistema consiglierà elementi che provengono da questi generi.

<br>
È importante notare che il content-based filtering è limitato dalla qualità e dalla quantità delle caratteristiche dell'elemento, se le caratteristiche non sono informative o pertinenti, le raccomandazioni non saranno accurate.
<br>

Il metodo **Content-based filtering** prevede in genere i seguenti passaggi:
- **Raccogliere e preelaborare le caratteristiche dell'oggetto**: il primo passo è raccogliere le caratteristiche degli oggetti, come il genere, l'artista, gli album, ecc.. e preelaborarle per l'analisi. Questo passaggio può includere la "pulizia" dei dati, la rimozione dei valori mancanti e la conversione di feature categoriche in valori numerici.
- **Creare un vettore di feature per ogni elemento**: una volta che le feature sono preelaborate, il passo successivo consiste nel creare un vettore di feature per ogni elemento. Questo vettore rappresenta le caratteristiche dell'elemento in forma numerica e viene utilizzato per confrontare la somiglianza tra gli elementi.
- **Calcolare la somiglianza tra gli elementi**: dopo aver creato i vettori di feature, il sistema utilizza una misura di somiglianza come la somiglianza del coseno, la somiglianza di Jaccard o la distanza euclidea per calcolare la somiglianza tra gli elementi.
- **Fare raccomandazioni**: una volta calcolata la somiglianza tra gli elementi, il sistema può formulare raccomandazioni agli utenti in base alla somiglianza. Ad esempio, se un utente ha mostrato interesse per un elemento specifico, il sistema può raccomandare altri elementi simili all'utente.
- **Rivalutare e aggiornare il modello**: il passo finale è rivalutare le prestazioni del modello e aggiornarlo se necessario. Questo passaggio può includere l'aggiunta di nuovi elementi, la rimozione di elementi irrilevanti e la regolazione del peso delle feature.

<br>

Vale la pena notare che il metodo di filtraggio basato sui contenuti può essere migliorato includendo informazioni aggiuntive, come le informazioni demografiche dell'utente, o integrandolo con altre tecniche di raccomandazione, come il Collaborative filtering.<br>
<br>
Inoltre, mentre il Content-based filtering è adatto per raccomandare elementi agli utenti con una storia di interazioni, non è bravo a raccomandare elementi a nuovi utenti senza alcuna cronologia, a questo scopo, può essere utilizzata una tecnica di filtraggio ibrido che combina i punti di forza sia del filtraggio basato sui contenuti che del filtraggio collaborativo.

In [8]:
print('Creo le tabelle pivot...')

# Creo una tabella pivot con il numero di tracks per playlist di ogni utente
group.create_tracks_pivot_table()

# Creo una tabella pivot con il numero di tracks per album di ogni utente
group.create_albums_pivot_table()

# Creo una tabella pivot 
group.create_artists_pivot_table()

Creo le tabelle pivot...


In [9]:
# Prendi i top 5 artisti e cerca le loro 2 canzoni più in voga

limit = 5
take_tracks = 2
i = 0

for key, row in group.get_artists_pivot_table()[:limit].iterrows():

    # Cerco le top tracks dell'artista
    result = sp.artist_top_tracks(artist_id=key)

    # Converto la risposta in json
    items_raw = json.dumps(result['tracks'])
    items_json = json.loads(items_raw)
    items_list = list(items_json)

    # Creo una lista temporanea delle tracks dell'artista corrente
    artist_tracks = list(map(lambda track: {
        'album_id': track.get('album', {}).get('id'),
        'album_name': track.get('album', {}).get('name'),
        'album_release_date': track.get('album', {}).get('release_date'),
        'artist_id': track.get('artists', {})[0].get('id'),
        'artist_name': track.get('artists', {})[0].get('name'),
        'track_disc_number': track.get('disc_number'),
        'track_duration_ms': track.get('duration_ms'),
        'track_explicit': track.get('explicit'),
        'track_popularity': track.get('popularity'),
        'track_id': track.get('id'),
        'track_name': track.get('name')
    }, items_list))

    # Ordino la lista temporanea per l'attributo track_populatiry
    artist_tracks = sorted(artist_tracks, key=lambda track: track['track_popularity'], reverse=True)

    # Aggiungo la lista temporanea alle informazioni del gruppo
    group.append_to_artists_top_tracks(artist_tracks[:take_tracks])

    i += 1
    if i < limit:
        time.sleep(5)

In [10]:
# Prendi i top 5 album e cerca le loro 2 canzoni più in voga

limit = 5
take_tracks = 2
i = 0

for key, row in group.get_albums_pivot_table()[:limit].iterrows():

    # Cerco le top tracks dell'artista
    result = sp.album_tracks(album_id=key)

    # Converto la risposta in json
    items_raw = json.dumps(result['items'])
    items_json = json.loads(items_raw)
    items_list = list(items_json)

    # Creo una lista temporanea delle tracks dell'album corrente
    album_tracks = list(map(lambda track: {
        'album_id': None,
        'album_name': None,
        'album_release_date': None,
        'artist_id': track.get('artists', {})[0].get('id'),
        'artist_name': track.get('artists', {})[0].get('name'),
        'track_disc_number': track.get('disc_number'),
        'track_duration_ms': track.get('duration_ms'),
        'track_explicit': track.get('explicit'),
        'track_popularity': None,
        'track_id': track.get('id'),
        'track_name': track.get('name')
    }, items_list))

    # Aggiungo la lista temporanea alle informazioni del gruppo
    group.append_to_albums_top_tracks(album_tracks[:take_tracks])

    i += 1
    if i < limit:
        time.sleep(5)

In [11]:
# Definizione della playlist raccomandata

# Inizializzo la Recommended playlist
group.recommended_playlist = group.get_artists_top_tracks() + group.get_albums_top_tracks()

In [12]:
# Definisco la recommended tracks playlist

i = 0
limit = 40
__tracks_list = group.get_playlist_tracks_dataframe()

# Fintantoché la lista non raggiunge il limit desiderato
while length_hint(group.get_recommended_playlist()) < limit:

    # Prondo le informazioni della track
    track_id = group.get_tracks_pivot_table().iloc[i].name
    results = __tracks_list.loc[__tracks_list['track_id'] == track_id]
    item = results.iloc[0]

    track = {
        'album_id': item.get('album_id'),
        'album_name': item.get('album_name'),
        'album_release_date': item.get('album_release_date'),
        'artist_id': item.get('artist_id'),
        'artist_name': item.get('artist_name'),
        'track_disc_number': item.get('track_disc_number'),
        'track_duration_ms': item.get('track_duration_ms'),
        'track_explicit': item.get('track_explicit'),
        'track_popularity': item.get('track_popularity'),
        'track_id': item.get('track_id'),
        'track_name': item.get('track_name')
    }

    # Appendo l'item in lista
    group.append_to_recommend_playlist(track)

    i += 1


In [13]:
# Ottengo la data corrente
current_date_time = datetime.now()

# Formato la data corrente
date_format = current_date_time.strftime("%Y-%m-%d")

# Informazioni della playlist
playlist_name = 'Recommend playlist - {}'.format(date_format)
playlist_description = 'SMM'

user_id = group.get_from_list(0).user_id

# Creo la nuova playlist su Spotify e l'aggiungo a quelle dell'utente
result = sp.user_playlist_create(user=user_id, name=playlist_name, description=playlist_description, public=True)


In [14]:
# Creo una lista temporanea delle recommended tracks
track_list = list(map(lambda x: x.get('track_id'), group.get_recommended_playlist()))

playlist_id = result.get('id')

# Aggiungo tutte le recommended tracks alla playlist appena creata
sp.user_playlist_add_tracks(user=user_id, playlist_id=playlist_id, tracks=track_list)

{'snapshot_id': 'MyxkYjVkZGM0NDdiMzQxMWViNmI2NGUzN2QxNzcyOGI5OThhNmNhNDJi'}

## Metodi di Sistemi di raccomandazione a confronto


### Collaborative filtering

Tra i **sistemi di raccomandazione** è noto il metodo **Collaborative filtering** per formulare raccomandazioni agli utenti in base al loro comportamento passato e al comportamento di utenti simili.<br>

> Questo sistema utilizza per identificare modelli e somiglianze tra loro, quindi utilizza questi modelli per formulare raccomandazioni ai nuovi utenti.<br>
<br>
Per implementare questo sistema, il primo passo è raccogliere dati sul comportamento passato degli utenti. Questi dati vengono quindi utilizzati per creare una matrice di elementi utente che rappresenta le valutazioni che gli utenti hanno dato agli elementi. Successivamente, il sistema utilizza una misura di somiglianza (come la somiglianza coseno) per identificare utenti o elementi simili. Infine, il sistema utilizza gli utenti o gli elementi simili identificati per formulare raccomandazioni ai nuovi utenti.<br>
<br>
È importante notare che il filtro collaborativo è una tecnica basata sulla memoria, significa che l'algoritmo deve avere accesso a tutte le interazioni utente-elemento per generare raccomandazioni, ecco perché non è adatto per set di dati di grandi dimensioni. Inoltre, può essere influenzato dal problema del cold-start, significa che l'algoritmo non può raccomandare elementi ai nuovi utenti che non hanno ancora interagito con il sistema.<br>

### Content-based filtering vs Collaborative filtering

È stato scelto di implementare un algoritmo basato sul metodo di <b>Content-based filtering</b> perché si basa sull'analisi di un set di informazioni, dove è possibile che non tutti gli utenti hanno una "storia passata", ovvero che non hanno espresso opinione, alimentando così il set di dati. Mentre per il metodo <b>Collaborative filtering</b> è importante per ottenere un risultato accettabile, che tutti gli utenti partecipano nel raccolta dei dati.<br>
In questo caso, dunque, si presuppone che gli elementi scelti dall'algoritmo, possano piacere pure agli utenti che non hanno espresso opinione, in quanto, piacciono ad altri utenti della lista.

## Feedback sulla playlist

![Feedback](img/Feedback.png)
![Feedback1](img/Feedback1.png)
![Feedback2](img/Feedback2.png)